In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("stackoverflow_full.csv")

# Skapa en kopia av landskolumnen
df["Country_raw"] = df["Country"]

# Ta bort länder som förekommer mindre än två gånger
counts = df["Country_raw"].value_counts()
valid_countries = counts[counts >= 2].index
df = df[df["Country_raw"].isin(valid_countries)]

# Droppa kolumner (se rapport)
cols_to_drop = [
    "Gender",
    "MentalHealth",
    "Accessibility",
    "Unnamed: 0",
    "HaveWorkedWith",
]
df = df.drop(columns=df.columns.intersection(cols_to_drop))

# Se till att 'YearsCodePro' inte är större än 'YearsCode'
df = df[df["YearsCodePro"] <= df["YearsCode"]]

# Ta bort rader där 'Age' är '<35' men 'YearsCode' är >35
df = df.drop(df[(df["Age"] == "<35") & (df["YearsCode"] > 35)].index)

# Skapa en normaliserad lönkolumn
df["Country_original"] = df["Country"]
median_salary_by_country = df.groupby("Country")["PreviousSalary"].median()
df["PreviousSalary_norm"] = df.apply(
    lambda r: r["PreviousSalary"] / median_salary_by_country[r["Country"]],
    axis=1
)

# Droppa den första lönkolumnen
df = df.drop(columns="PreviousSalary")

# One-Hot Encoda kategoriska variabler
categorical_cols = ["Age", "EdLevel", "MainBranch", "Country", "Employment"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Skapa en grupperad landskolumn för stratifiering, baserat på den rensade listan av länder
counts_grouped = df["Country_raw"].value_counts()
rare_grouped = counts_grouped[counts_grouped < 2].index
df["Country_grouped"] = df["Country_raw"].replace(rare_grouped, "Other")

# Visa de första raderna av den färdiga datan
print("Antal rader efter rensning:", len(df))
print("Antal kolumner efter encoding:", len(df.columns))
df.head()




Antal rader efter rensning: 72859
Antal kolumner efter encoding: 175


,YearsCode,YearsCodePro,ComputerSkills,Employed,Country_raw,Country_original,PreviousSalary_norm,Age_>35,EdLevel_NoHigherEd,EdLevel_Other,...,Country_United States of America,Country_Uruguay,Country_Uzbekistan,"Country_Venezuela, Bolivarian Republic of...",Country_Viet Nam,Country_Yemen,Country_Zambia,Country_Zimbabwe,Employment_1,Country_grouped
0,7,4,4,0,Sweden,Sweden,0.881050,False,False,False,...,False,False,False,False,False,False,False,False,True,Sweden
1,12,5,12,1,Spain,Spain,1.089669,False,False,False,...,False,False,False,False,False,False,False,False,True,Spain
2,15,6,7,0,Germany,Germany,1.191662,False,False,False,...,False,False,False,False,False,False,False,False,True,Germany
3,9,6,13,0,Canada,Canada,0.590838,False,False,False,...,False,False,False,False,False,False,False,False,True,Canada
4,40,30,2,0,Singapore,Singapore,2.458366,True,False,False,...,False,False,False,False,False,False,False,False,False,Singapore
